In [146]:
import cv2
import os
import numpy as np
from tqdm import tqdm
from random import shuffle
from __future__ import division, print_function, absolute_import
# library for optmising inference

TRAIN_DIR= 'your training directory'

IMG_SIZE= 50
keep_rate=0.5
LR= 1e-3
MODEL_NAME= 'Automatic_Checker_System-{}-{}.model'.format(LR,'4__conv-basic')

In [89]:
def label_img(img):
    label=[0]*200
    word_label= img.split('-')[-2]
    label[int(word_label)-1]+=1
    return label

In [61]:
def create_train_data():
    training_data=[]
    src_files= os.listdir(TRAIN_DIR)
    for files in src_files:
        full_path= os.path.join(TRAIN_DIR,files)
        for img in tqdm(os.listdir(full_path)):
            label = label_img(img)
            path= os.path.join(full_path,img)
            img= cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE),interpolation=cv2.INTER_CUBIC)
            training_data.append([np.array(img),np.array(label)])
    
    shuffle(training_data)
    np.save('train_data.npy',training_data)
    return training_data

In [62]:
train_data= create_train_data()

100%|██████████████████████████████████████████| 73/73 [00:01<00:00, 41.09it/s]


In [147]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow as tf
tf.reset_default_graph()


convnet= input_data(shape=[None,IMG_SIZE,IMG_SIZE,1], name= 'input')

convnet= conv_2d(convnet,32,2, activation='relu')
convnet= max_pool_2d(convnet,2)

convnet= conv_2d(convnet,64,2, activation='relu')
convnet= max_pool_2d(convnet,2)

convnet= conv_2d(convnet,96,2, activation='relu')
convnet= max_pool_2d(convnet,2)

convnet= conv_2d(convnet,128,2, activation='relu')
convnet= max_pool_2d(convnet,2)



convnet= fully_connected(convnet, 200 , activation='softmax')

convnet= regression(convnet, optimizer='adam', learning_rate=LR,loss='categorical_crossentropy', name='targets')

model =tflearn.DNN(convnet, tensorboard_dir='log')


In [148]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')
print(len(train_data))

INFO:tensorflow:Restoring parameters from C:\Users\Dev\MachineLearningProjects\Automatic_Checker_System-0.001-4__conv-basic.model
model loaded!
14443


In [151]:
train= train_data[:-800]
test= train_data[-800:]

In [111]:
X= np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y= [i[1] for i in train]

test_x= np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y= [i[1] for i in test]


In [132]:
model.fit({'input':X},{'targets':Y}, n_epoch=20, validation_set=({'input':test_x},{'targets':test_y}), snapshot_step=500, show_metric=True, run_id=MODEL_NAME)


Training Step: 49  | total loss: nan | time: 10.138s
| Adam | epoch: 001 | loss: nan - acc: 0.0063 -- iter: 01600/13643


KeyboardInterrupt: 

In [153]:
model.save(MODEL_NAME)

INFO:tensorflow:C:\Users\Dev\MachineLearningProjects\Automatic_Checker_System-0.001-4__conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.
